Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
         Mean         SD       Naive SE       MCSE     ESS
alpha 154.600221 0.273999474 0.0043323121 0.004594271 1000
 beta   0.904589 0.041345051 0.0006537227 0.000778469 1000
sigma   5.101196 0.191555673 0.0030287611 0.003079221 1000

Quantiles:
          2.5%        25.0%      50.0%      75.0%       97.5%   
alpha 154.06700000 154.412000 154.601000 154.7850000 155.1320000
 beta   0.82585217   0.876491   0.904441   0.9321370   0.9859226
sigma   4.74837075   4.970615   5.095080   5.2275125   5.4826422



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.769 0.893227 5.14041; 154.824 0.869991 4.96286; … ; 154.297 0.89518 4.76915; 154.548 0.920248 4.79085]

Union{Missing, Float64}[154.345 0.932257 5.03306; 154.659 0.934779 5.40642; … ; 154.582 0.867129 4.98089; 154.671 0.910674 5.1329]

Union{Missing, Float64}[154.472 0.95168 5.08019; 154.607 0.96064 5.25558; … ; 154.265 0.9225 5.30344; 154.636 0.880053 4.85975]

Union{Missing, Float64}[154.105 0.944031 5.07175; 154.87 0.855622 5.06342; … ; 154.791 0.863154 4.94186; 154.749 0.903901 4.91117]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
         Mean         SD       Naive SE       MCSE     ESS
alpha 154.600221 0.273999474 0.0043323121 0.004594271 1000
 beta   0.904589 0.041345051 0.0006537227 0.000778469 1000
sigma   5.101196 0.191555673 0.0030287611 0.003079221 1000

Quantiles:
          2.5%        25.0%      50.0%      75.0%       97.5%   
alpha 154.06700000 154.412000 154.601000 154.7850000 155.1320000
 beta   0.82585217   0.876491   0.904441   0.9321370   0.9859226
sigma   4.74837075   4.970615   5.095080   5.2275125   5.4826422



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*